# Laboratorium 2 - Analiza głównych składowych

## Preprocessing danych

In [178]:
import os
import numpy as np
from PIL import Image

In [179]:
def load_images_from_folders(data_folders):
    images = []
    for folder in data_folders:
        folder_path = os.path.join("data", folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            images.append(img)
    return images

In [180]:
def convert_pictures(images):
    for i, img in enumerate(images):
        images[i] = img.convert('L')
        images[i] = images[i].resize((224, 224))
        images[i] = np.array(images[i])
        images[i] = np.reshape(images[i], -1)
        images[i] = images[i] - np.mean(images[i])


In [181]:
data_folders = ["mold", "mosaic", "septoria"]

In [182]:
images = load_images_from_folders(data_folders)
convert_pictures(images)
X = np.vstack(images)

print("Wymiary tablicy X ->", X.shape)

Wymiary tablicy X -> (60, 50176)
